In [126]:
import pandas as pd
import geopandas as gpd
import numpy as np
import networkx as nx

pd.options.display.max_columns = None
pd_max_colwidth_original = pd.options.display.max_colwidth
# pd.options.display.max_colwidth = None
gpd.options.io_engine = "pyogrio"

In [109]:
faf_nodes_df = gpd.read_file(r'zip://FAF5_Model_Highway_Network.zip!Networks/Geodatabase Format/FAF5Network.gdb/a0000000a.gdbtable')
print(f'Number of nodes prior to drop duplicates: {len(faf_nodes_df)}')
faf_nodes_df.drop_duplicates(inplace=True)
faf_nodes_df.drop(faf_nodes_df[faf_nodes_df.State.isin(["HI", "AK"])].index, inplace=True)
faf_nodes_df.reset_index(drop=True, inplace=True)
print(f'Number of nodes after to drop duplicates: {len(faf_nodes_df)}')
faf_links_df = gpd.read_file(r'zip://FAF5_Model_Highway_Network.zip!Networks/Geodatabase Format/FAF5Network.gdb/a00000009.gdbtable')
faf_links_df.drop(faf_links_df[faf_links_df.STATE.isin(["HI", "AK"])].index, inplace=True)
faf_links_df.reset_index(drop=True, inplace=True)

Number of nodes prior to drop duplicates: 974788
Number of nodes after to drop duplicates: 348442


In [112]:
faf_nodes_df.tail(2)

,ID,DATA,Entry_or_Exit,Exit_Number,Interchange,Centroid,CentroidID,Facility_Type,Facility_Name,County,State,StateID,StateName,FAFID,StateNameBak,geometry
348440,1949490,46492932,None,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,POINT (-68.76264 44.80220)
348441,1949491,46492239,None,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,POINT (-68.76740 44.80308)


In [119]:
faf_links_df.tail(2)

,ID,LENGTH,DIR,DATA,VERSION,Class,Class_Description,Road_Name,Sign_Rte,Rte_Type,Rte_Number,Rte_Qualifier,Country,STATE,STFIPS,County_Name,CTFIPS,Urban_Code,FAFZONE,Status,F_Class,Facility_Type,NHS,STRAHNET,NHFN,Truck,AB_Lanes,BA_Lanes,Speed_Limit,Toll_Type,Toll_Name,Toll_Link,Toll_Link_Name,HPMS_USA_RouteID,HPMS_Begin_Point,HPMS_End_Point,BorderState1,BorderState2,BorderFAF1,BorderFAF2,TRUCKTOLL,BorderLink,AddedBorderTime,AdjustSpeed,AdjustReason,AB_FinalSpeed,BA_FinalSpeed,AB_CombinedSpeed,BA_CombinedSpeed,AB_FreeFlowTime,BA_FreeFlowTime,SHAPE_Length,geometry,FROM_NODE_IDX,TO_NODE_IDX
484444,1949503,0.168653,0,669923,V2021.05,14.0,Arterial or Major Collector,BROADWAY,ME 15,S,15,None,USA,ME,23,PENOBSCOT,23019,04951,230.0,1.0,3.0,2.0,10.0,NaN,NaN,None,2.0,2.0,25.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,25.0,25.0,25.0,25.0,0.404766,0.404766,0.002789,"MULTILINESTRING ((-68.77395 44.81775, -68.7743...",348322,348319
484445,1949504,0.207630,1,775835,V2021.05,22.0,Ramp,RAMP,None,None,None,None,USA,ME,23,PENOBSCOT,23019,04951,230.0,1.0,1.0,4.0,12.0,NaN,NaN,None,1.0,NaN,35.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,29.5,29.5,29.5,29.5,0.422299,0.422299,0.003698,"MULTILINESTRING ((-68.77395 44.81775, -68.7737...",348322,348321


In [82]:
print(f'faf_links_df.csr: {faf_links_df.crs}')
print(f'faf_nodes_df.csr: {faf_nodes_df.crs}')
print(f'faf_links_df.shape: {faf_links_df.shape}')
print(f'faf_nodes_df.shape: {faf_nodes_df.shape}')
print(f'len(faf_links_df): {len(faf_links_df)}')
print(f'len(faf_nodes_df): {len(faf_nodes_df)}')
print(f'len(faf_links_df.columns): {len(faf_links_df.columns)}')
print(f'len(faf_nodes_df.columns): {len(faf_nodes_df.columns)}')

faf_links_df.csr: EPSG:4269
faf_nodes_df.csr: EPSG:4269
faf_links_df.shape: (484446, 53)
faf_nodes_df.shape: (348442, 16)
len(faf_links_df): 484446
len(faf_nodes_df): 348442
len(faf_links_df.columns): 53
len(faf_nodes_df.columns): 16


In [95]:
import shapely

In [117]:
node_lat_long_to_idx = {(row.geometry.coords[0][1], row.geometry.coords[0][0]): row.Index for row in faf_nodes_df.itertuples(index=True)}
len(node_lat_long_to_idx), len(faf_nodes_df)

(348442, 348442)

In [118]:
from_node_idx: list[int] = list()
to_node_idx: list[int] = list()
links_with_no_valid_nodes: list[tuple[int, str, str, str, int, int]] = list()
for row in faf_links_df.itertuples(index=True):
    if shapely.get_num_geometries(row.geometry) > 1:
        print(row)
    linestring = shapely.get_geometry(row.geometry, 0)
    one_end_point = (linestring.coords[0][1], linestring.coords[0][0])
    other_end_point = (linestring.coords[-1][1], linestring.coords[-1][0])
    one_end_point_idx = node_lat_long_to_idx[one_end_point] if one_end_point in node_lat_long_to_idx else -1
    other_end_point_idx = node_lat_long_to_idx[other_end_point] if other_end_point in node_lat_long_to_idx else -1
    from_node_idx.append(one_end_point_idx)
    to_node_idx.append(other_end_point_idx)
       
    if one_end_point_idx == -1 or other_end_point_idx == -1:
        links_with_no_valid_nodes.append((row.ID, row.Country, row.STATE, row.Road_Name, one_end_point_idx, other_end_point_idx))
    
links_with_no_valid_nodes
faf_links_df['FROM_NODE_IDX'] = from_node_idx
faf_links_df['TO_NODE_IDX'] = to_node_idx

In [151]:
len(set(from_node_idx) | set(to_node_idx))

346168

In [134]:
nx_highway_graph = nx.DiGraph()

edge_idx_counter = 0
for row in faf_links_df.itertuples():

    origin_node_idx = row.FROM_NODE_IDX
    destination_node_idx = row.TO_NODE_IDX
    if origin_node_idx == -1 or destination_node_idx == -1:
        continue

    origin_node = faf_nodes_df.loc[origin_node_idx]
    destination_node = faf_nodes_df.loc[destination_node_idx]

    # add the nodes with the right attributese to be used by dyntapy
    nx_highway_graph.add_node(
        origin_node_idx, x_coord=origin_node.geometry.coords[0][0], y_coord=origin_node.geometry.coords[0][1])
    nx_highway_graph.add_node(
        destination_node_idx, x_coord=destination_node.geometry.coords[0][0], y_coord=destination_node.geometry.coords[0][1])

    # add the edges with the right attributes to be used by dyntapy
    # ‘from_node_id’, ‘to_node_id’, ‘link_id’, ‘lanes’, ‘capacity’, ‘length’, ‘free_speed’
    length_miles = row.LENGTH
    ab_lanes = 0 if (np.isnan(row.AB_Lanes) or row.AB_Lanes <
                     1) else row.AB_Lanes  # many links have NaN or 0 lanes
    ba_lanes = 0 if (np.isnan(row.BA_Lanes) or row.BA_Lanes < 1) else row.BA_Lanes
    ab_edge = {
        'u_of_edge': origin_node_idx,
        'v_of_edge': destination_node_idx,
        'from_node_id': origin_node_idx,
        'to_node_id': destination_node_idx,
        'lanes': ab_lanes,
        'length': length_miles,
        'free_speed': row.AB_FinalSpeed,  # still not sure of this
        'capacity': ab_lanes*22000
    }
    ba_edge = {
        'u_of_edge': destination_node_idx,
        'v_of_edge': origin_node_idx,
        'from_node_id': destination_node_idx,
        'to_node_id': origin_node_idx,
        'lanes': ba_lanes,
        'length': length_miles,
        'free_speed': row.BA_FinalSpeed,  # still not sure of this
        'capacity': ba_lanes*22000
    }

    if row.DIR == 1:  # A-> B only
        ab_edge["link_id"] = edge_idx_counter
        nx_highway_graph.add_edge(**ab_edge)
        edge_idx_counter += 1

    elif row.DIR == -1:  # B->A only
        ba_edge["link_id"] = edge_idx_counter
        nx_highway_graph.add_edge(**ba_edge)
        edge_idx_counter += 1

    else:
        # add A->B
        ab_edge["link_id"] = edge_idx_counter
        nx_highway_graph.add_edge(**ab_edge)
        edge_idx_counter += 1

        # add B->A
        ba_edge["link_id"] = edge_idx_counter
        nx_highway_graph.add_edge(**ba_edge)
        edge_idx_counter += 1

print(f"edges {edge_idx_counter:,}")

edges 651,072


In [137]:
print(f"Is graph strongly connected: {nx.is_strongly_connected(nx_highway_graph)}")
print(f"Number of strongly connected components {len([x for x in nx.strongly_connected_components(nx_highway_graph)]):,}")
print(f"Number of weekly connected components {len([x for x in nx.weakly_connected_components(nx_highway_graph)]):,}")

Is graph strongly connected: False
Number of strongly connected components 14,942
Number of weekly connected components 152


In [145]:
s_components = [x for x in nx.strongly_connected_components(nx_highway_graph)]
[ for x in s_components if len(x) > 1]

[10, 9, 2, 2, 4, 2, 331177, 2, 6, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 5, 2]

In [148]:
s_components[10000]

{120178}

In [150]:
lon, lat = faf_nodes_df.loc[120178].geometry.coords[0]
lat, lon

(32.02725600000002, -93.53400999999997)

In [136]:
fake_graph = nx.DiGraph([(0, 1), (1, 2), (2, 3)])
nx.is_strongly_connected(fake_graph)

False

In [49]:
faf_nodes_df.loc[[892231,892233]]

,ID,DATA,Entry_or_Exit,Exit_Number,Interchange,Centroid,CentroidID,Facility_Type,Facility_Name,County,State,StateID,StateName,FAFID,StateNameBak,geometry
892231,1784580,46786560,None,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,POINT (-75.43572 40.08475)
892233,1784581,46786566,Exit,326,VALLEY FORGE,NaN,NaN,Toll Road Exit Point,PENNSYLVANIA TURNPIKE-TICKET SYSTEM,None,None,NaN,None,NaN,None,POINT (-75.41299 40.09064)


In [67]:
x = faf_links_df.loc[faf_links_df.ID.isin([1784578, 1784579, 1391622])]
x

,ID,LENGTH,DIR,DATA,VERSION,Class,Class_Description,Road_Name,Sign_Rte,Rte_Type,Rte_Number,Rte_Qualifier,Country,STATE,STFIPS,County_Name,CTFIPS,Urban_Code,FAFZONE,Status,F_Class,Facility_Type,NHS,STRAHNET,NHFN,Truck,AB_Lanes,BA_Lanes,Speed_Limit,Toll_Type,Toll_Name,Toll_Link,Toll_Link_Name,HPMS_USA_RouteID,HPMS_Begin_Point,HPMS_End_Point,BorderState1,BorderState2,BorderFAF1,BorderFAF2,TRUCKTOLL,BorderLink,AddedBorderTime,AdjustSpeed,AdjustReason,AB_FinalSpeed,BA_FinalSpeed,AB_CombinedSpeed,BA_CombinedSpeed,AB_FreeFlowTime,BA_FreeFlowTime,SHAPE_Length,geometry
347879,1391622,4.504087,1,874749,V2021.05,11.0,Interstate Highway,PENNSYLVANIA TPKE,I 76,I,76,None,USA,PA,42,CHESTER,42029,69076,421.0,1.0,1.0,2.0,1.0,1.0,1.0,None,3.0,NaN,65.0,1.0,PENNSYLVANIA TURNPIKE,NaN,None,PA_2 15 7076 - 11344,19.931460,24.436850,None,None,NaN,NaN,0.585531,NaN,NaN,NaN,None,58.5,58.5,58.5,58.5,4.619577,4.619577,0.084367,"MULTILINESTRING ((-75.52663 40.07413, -75.5258..."
446115,1784578,0.416432,1,875421,V2021.05,11.0,Interstate Highway,PENNSYLVANIA TPKE,I 76,I,76,None,USA,PA,42,CHESTER,42029,69076,421.0,1.0,1.0,2.0,1.0,1.0,1.0,None,2.0,NaN,65.0,1.0,PENNSYLVANIA TURNPIKE,NaN,None,PA_2 15 7076 - 11344,24.436850,24.853413,None,None,NaN,NaN,0.054136,NaN,NaN,NaN,None,58.5,58.5,58.5,58.5,0.427110,0.427110,0.007690,"MULTILINESTRING ((-75.44316 40.08281, -75.4430..."
446116,1784579,1.271356,1,875425,V2021.05,11.0,Interstate Highway,PENNSYLVANIA TPKE,I 76,I,76,None,USA,PA,42,CHESTER,42029,69076,421.0,1.0,1.0,2.0,1.0,1.0,1.0,None,3.0,NaN,65.0,1.0,PENNSYLVANIA TURNPIKE,NaN,None,PA_2 15 7076 - 11344,24.853413,26.030000,None,None,NaN,NaN,0.165276,NaN,NaN,NaN,None,58.5,58.5,58.5,58.5,1.303954,1.303954,0.023482,"MULTILINESTRING ((-75.43572 40.08475, -75.4348..."


In [70]:
faf_links_df.DIR.value_counts()

DIR
 1    318788
 0    168215
-1       391
Name: count, dtype: int64

In [123]:
faf_nodes_df.loc[faf_links_df.loc[443800].FROM_NODE_IDX]

ID                                            1781536
DATA                                         47022483
Entry_or_Exit                                    None
Exit_Number                                      None
Interchange                                      None
Centroid                                          NaN
CentroidID                                        NaN
Facility_Type                                    None
Facility_Name                                    None
County                                           None
State                                            None
StateID                                           NaN
StateName                                        None
FAFID                                             NaN
StateNameBak                                     None
geometry         POINT (-75.103902 40.03057800000005)
Name: 320279, dtype: object

In [124]:
faf_nodes_df.loc[faf_links_df.loc[443800].TO_NODE_IDX]

ID                                             1786931
DATA                                          47022481
Entry_or_Exit                                     None
Exit_Number                                       None
Interchange                                       None
Centroid                                           NaN
CentroidID                                         NaN
Facility_Type                                     None
Facility_Name                                     None
County                                            None
State                                             None
StateID                                            NaN
StateName                                         None
FAFID                                              NaN
StateNameBak                                      None
geometry         POINT (-75.103186 40.030054000000064)
Name: 321132, dtype: object

In [120]:
faf_links_df[(faf_links_df.DIR == -1) & (faf_links_df.STATE == "PA")]

,ID,LENGTH,DIR,DATA,VERSION,Class,Class_Description,Road_Name,Sign_Rte,Rte_Type,Rte_Number,Rte_Qualifier,Country,STATE,STFIPS,County_Name,CTFIPS,Urban_Code,FAFZONE,Status,F_Class,Facility_Type,NHS,STRAHNET,NHFN,Truck,AB_Lanes,BA_Lanes,Speed_Limit,Toll_Type,Toll_Name,Toll_Link,Toll_Link_Name,HPMS_USA_RouteID,HPMS_Begin_Point,HPMS_End_Point,BorderState1,BorderState2,BorderFAF1,BorderFAF2,TRUCKTOLL,BorderLink,AddedBorderTime,AdjustSpeed,AdjustReason,AB_FinalSpeed,BA_FinalSpeed,AB_CombinedSpeed,BA_CombinedSpeed,AB_FreeFlowTime,BA_FreeFlowTime,SHAPE_Length,geometry,FROM_NODE_IDX,TO_NODE_IDX
377176,1520443,0.085173,-1,552515,V2021.05,14.0,Arterial or Major Collector,PA 68,PA 68,S,68,None,USA,PA,42,BEAVER,42007,69697,422.0,1.0,3.0,6.0,12.0,NaN,NaN,None,1.0,NaN,50.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,43.0,43.0,43.0,43.0,0.118846,0.118846,0.001579,"MULTILINESTRING ((-80.29643 40.69873, -80.2963...",274800,274805
382130,1540373,0.015269,-1,554440,V2021.05,14.0,Arterial or Major Collector,MIFFLIN RD,PA 885,S,885,None,USA,PA,42,ALLEGHENY,42003,69697,422.0,1.0,3.0,2.0,10.0,NaN,NaN,None,4.0,NaN,45.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,38.5,38.5,38.5,38.5,0.023796,0.023796,0.000222,"MULTILINESTRING ((-79.93336 40.39000, -79.9333...",278358,278348
382162,1540405,0.214107,-1,1358727,V2021.05,14.0,Arterial or Major Collector,BEECHWOOD BLVD,None,None,None,None,USA,PA,42,ALLEGHENY,42003,69697,422.0,1.0,3.0,2.0,10.0,NaN,NaN,None,2.0,1.0,50.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,43.0,43.0,43.0,43.0,0.298754,0.298754,0.003380,"MULTILINESTRING ((-79.92699 40.42490, -79.9269...",278380,278377
383731,1546774,0.026513,-1,553947,V2021.05,14.0,Arterial or Major Collector,FORT PITT BLVD,PA 885,S,885,None,USA,PA,42,ALLEGHENY,42003,69697,422.0,1.0,3.0,6.0,11.0,NaN,NaN,None,2.0,2.0,45.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,38.5,38.5,38.5,38.5,0.041318,0.041318,0.000422,"MULTILINESTRING ((-80.00565 40.43823, -80.0057...",278897,279401
416204,1676591,0.102144,-1,874977,V2021.05,14.0,Arterial or Major Collector,COLONEL HOWARD BLVD,None,None,None,None,USA,PA,42,LANCASTER,42071,47530,429.0,1.0,3.0,2.0,12.0,NaN,NaN,None,NaN,NaN,45.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,38.5,38.5,38.5,38.5,0.159185,0.159185,0.001931,"MULTILINESTRING ((-76.08247 40.21460, -76.0844...",302128,302124
442851,1783302,0.031252,-1,616540,V2021.05,14.0,Arterial or Major Collector,NEW STATE RD,PA 73,S,73,None,USA,PA,42,PHILADELPHIA,42101,69076,421.0,1.0,3.0,6.0,11.0,NaN,NaN,None,2.0,2.0,45.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,38.5,38.5,38.5,38.5,0.048705,0.048705,0.000569,"MULTILINESTRING ((-75.04832 40.01771, -75.0482...",320546,320547
442880,1783331,0.112678,-1,1123909,V2021.05,16.0,Frontage/Service Road,ROOSEVELT BLVD FRONTAGE RD,US 1,U,1,None,USA,PA,42,PHILADELPHIA,42101,69076,421.0,1.0,3.0,1.0,10.0,NaN,NaN,None,6.0,NaN,45.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,38.5,38.5,38.5,38.5,0.175602,0.175602,0.001831,"MULTILINESTRING ((-75.05411 40.04431, -75.0554...",320564,320563
443800,1786939,0.052449,-1,1123948,V2021.05,14.0,Arterial or Major Collector,ADAMS AVE,None,None,None,None,USA,PA,42,PHILADELPHIA,42101,69076,421.0,1.0,3.0,1.0,10.0,NaN,NaN,None,2.0,2.0,45.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,38.5,38.5,38.5,38.5,0.081739,0.081739,0.000888,"MULTILINESTRING ((-75.10390 40.03058, -75.1037...",320279,321132
443944,1787659,0.127249,-1,615727,V2021.05,14.0,Arterial or Major Collector,ARAMINGO AVE,None,None,None,None,USA,PA,42,PHILADELPHIA,42101,69076,421.0,1.0,3.0,6.0,11.0,NaN,NaN,None,2.0,2.0,50.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,43.0,43.0,43.0,43.0,0.177557,0.177557,0.001851,"MULTILINESTRING ((-75.12331 39.97069, -75.1233...",321221,321229
443946,1787661,0.052770,-1,615718,V2021.05,14.0,A

In [68]:
x.explore()

In [53]:
faf_nodes_df.loc[[889753,889751]]

,ID,DATA,Entry_or_Exit,Exit_Number,Interchange,Centroid,CentroidID,Facility_Type,Facility_Name,County,State,StateID,StateName,FAFID,StateNameBak,geometry
889753,1779506,46788743,None,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,POINT (-75.21306 40.17764)
889751,1779504,46788691,None,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,POINT (-75.20136 40.15929)


In [55]:
faf_links_df.loc[[444875,444881]]

,ID,LENGTH,DIR,DATA,VERSION,Class,Class_Description,Road_Name,Sign_Rte,Rte_Type,Rte_Number,Rte_Qualifier,Country,STATE,STFIPS,County_Name,CTFIPS,Urban_Code,FAFZONE,Status,F_Class,Facility_Type,NHS,STRAHNET,NHFN,Truck,AB_Lanes,BA_Lanes,Speed_Limit,Toll_Type,Toll_Name,Toll_Link,Toll_Link_Name,HPMS_USA_RouteID,HPMS_Begin_Point,HPMS_End_Point,BorderState1,BorderState2,BorderFAF1,BorderFAF2,TRUCKTOLL,BorderLink,AddedBorderTime,AdjustSpeed,AdjustReason,AB_FinalSpeed,BA_FinalSpeed,AB_CombinedSpeed,BA_CombinedSpeed,AB_FreeFlowTime,BA_FreeFlowTime,SHAPE_Length,geometry
444875,1779498,0.764464,1,877908,V2021.05,12.0,Other Controlled Access Highway,SR 309,SR 309,S,309,None,USA,PA,42,MONTGOMERY,42091,69076,421.0,1.0,2.0,6.0,11.0,NaN,NaN,None,2.0,NaN,65.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,56.5,56.5,56.5,56.5,0.811820,0.811820,0.011802,"MULTILINESTRING ((-75.20684 40.16917, -75.2068..."
444881,1779504,0.366698,1,877834,V2021.05,12.0,Other Controlled Access Highway,SR 309,SR 309,S,309,None,USA,PA,42,MONTGOMERY,42091,69076,421.0,1.0,2.0,6.0,11.0,NaN,NaN,None,3.0,NaN,65.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,56.5,56.5,56.5,56.5,0.389414,0.389414,0.005564,"MULTILINESTRING ((-75.19916 40.10903, -75.1991..."
